In [1]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal

class FullBayes:
    def __init__(self, filename):
        self.df = pd.read_csv(filename, sep=',', header=None)
        self.df.columns = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width','class']
        
        self.n_points = int(self.df.shape[0] * 0.7)
        # Bootstrap
        train_indices = np.random.randint(self.df.shape[0], size=self.n_points)
        self.df_train = self.df.iloc[train_indices]
        # Test points
        test_indices = np.setdiff1d(np.arange(self.df.shape[0]), train_indices)
        self.df_test = self.df.iloc[test_indices]
        
        gb = self.df_train.groupby('class')
        self.group_names = list(gb.groups)
        self.groups = [gb.get_group(x) for x in gb.groups]
        self.n_points_per_group = [x.shape[0] for x in self.groups]
        self.prior_per_group = [x / (self.n_points + 0.0) for x in self.n_points_per_group]
        self.mean_per_group = [x.mean(numeric_only=True) for x in self.groups]
        self.centered_groups = [x - self.mean_per_group[i] for i, x in enumerate(self.groups)]
        self.cov_per_group = [x.cov() for x in self.groups]
    
    def fit(self):
        return self.prior_per_group, self.mean_per_group, self.cov_per_group
    
    def predict(self):
        X = self.df_test[['sepal-length', 'sepal-width', 'petal-length', 'petal-width']]
        y_true = self.df_test[['class']].squeeze()

        res = np.empty((X.shape[0], len(self.cov_per_group)))
        for i, el in enumerate(self.cov_per_group):
            cov = self.cov_per_group[i].to_numpy()
            mean = self.mean_per_group[i].to_numpy()
            normal = multivariate_normal.pdf(X, mean=mean, cov=cov) * self.prior_per_group[i]
            res[:, i] = normal
        pred = np.argmax(res, axis=1)
        y_pred = pd.DataFrame(pred).apply(lambda x: self.group_names[x[0]], axis=1, raw=True)
        n_errors = np.count_nonzero(np.not_equal(y_pred.to_numpy(), y_true.to_numpy()))
        return y_pred, y_true, n_errors / (0.0 + y_true.shape[0])
        
    
    def score(self):
        _, _, error_rate = self.predict()
        return error_rate
        
        

In [2]:
fBayes = FullBayes('iris.data')
fBayes.score()

0.0

In [3]:
import pandas as pd
from scipy.stats import multivariate_normal

class NaiveBayes:
    def __init__(self, filename):
        self.df = pd.read_csv(filename, sep=',', header=None)
        self.df.columns = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width','class']
        
        self.n_points = int(self.df.shape[0] * 0.7)
        # Bootstrap
        train_indices = np.random.randint(self.df.shape[0], size=self.n_points)
        self.df_train = self.df.iloc[train_indices]
        # Test points
        test_indices = np.setdiff1d(np.arange(self.df.shape[0]), train_indices)
        self.df_test = self.df.iloc[test_indices]
        
        gb = self.df.groupby('class')
        self.group_names = list(gb.groups)
        self.groups = [gb.get_group(x) for x in gb.groups]
        self.n_points_per_group = [x.shape[0] for x in self.groups]
        self.prior_per_group = [x / (self.n_points + 0.0) for x in self.n_points_per_group]
        self.mean_per_group = [x.mean(numeric_only=True) for x in self.groups]
        self.centered_groups = [x - self.mean_per_group[i] for i, x in enumerate(self.groups)]
        self.cov_per_group = [x.cov() for x in self.groups]
    
    def fit(self):
        return self.prior_per_group, self.mean_per_group, self.cov_per_group
    
    def predict(self):
        X = self.df_test[['sepal-length', 'sepal-width', 'petal-length', 'petal-width']].to_numpy()
        n_attributes = X.shape[1]
        y_true = self.df_test[['class']].squeeze()

        res = np.empty((X.shape[0], len(self.cov_per_group)))
        for i, el in enumerate(self.cov_per_group):
            cov = self.cov_per_group[i].to_numpy()
            mean = self.mean_per_group[i].to_numpy()
            prior = self.prior_per_group[i]
            normal = np.ones(X.shape[0])
            for j in range(n_attributes):
                normal = normal * multivariate_normal.pdf(X[:, j], mean=mean[j], cov=cov[j, j])
            res[:, i] = normal * prior
        pred = np.argmax(res, axis=1)
        y_pred = pd.DataFrame(pred).apply(lambda x: self.group_names[x[0]], axis=1, raw=True)
        n_errors = np.count_nonzero(np.not_equal(y_pred.to_numpy(), y_true.to_numpy()))
        return y_pred, y_true, n_errors / (0.0 + y_true.shape[0])
        
    
    def score(self):
        _, _, error_rate = self.predict()
        return error_rate
        
        

In [4]:
fBayes = NaiveBayes('iris.data')
fBayes.score()

0.02564102564102564

In [5]:
def Paired_T_Test(t_alpha_half, K):
    delta = np.empty(K)
    for i in range(K):
        score_NB = NaiveBayes('iris.data').score()
        score_FB = FullBayes('iris.data').score()
        delta[i] = score_NB - score_FB
    mean = delta.mean()
    std = delta.std()
    Z = (np.sqrt(K) * mean) / std
    print('z-score: ', Z)
    if Z >= -t_alpha_half and Z <= t_alpha_half:
        return 'Accept H0: both classifires has similar performance'
    
    return 'Reject H0: classifires have significantly different performance'

#https://www.stat.colostate.edu/inmem/gumina/st201/pdf/Utts-Heckard_t-Table.pdf

for i in range(10):
    #95%
    r = Paired_T_Test(2.05, 30)
    print('95% conf: ', r)
    #99%
    r = Paired_T_Test(2.76, 30)
    print('99% conf: ', r)

z-score:  -0.085610608506712
95% conf:  Accept H0: both classifires has similar performance
z-score:  2.0657124051604376
99% conf:  Accept H0: both classifires has similar performance
z-score:  4.451950976612078
95% conf:  Reject H0: classifires have significantly different performance
z-score:  2.2381042041280463
99% conf:  Accept H0: both classifires has similar performance
z-score:  1.6579297369548605
95% conf:  Accept H0: both classifires has similar performance
z-score:  0.5274646052085252
99% conf:  Accept H0: both classifires has similar performance
z-score:  0.7317809085107079
95% conf:  Accept H0: both classifires has similar performance
z-score:  0.6728210322911085
99% conf:  Accept H0: both classifires has similar performance
z-score:  2.6809767306850008
95% conf:  Reject H0: classifires have significantly different performance
z-score:  1.7256488921623931
99% conf:  Accept H0: both classifires has similar performance
z-score:  2.6469332155844114
95% conf:  Reject H0: classi